In [31]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.svm import SVR, SVC

from sklearn.metrics import mean_squared_error, f1_score
from sklearn.metrics import mean_squared_error

In [32]:
# Генерируем уникальный seed
my_code = "Железов"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [33]:
# Читаем данные из файла
example_data = pd.read_csv("datasets/Fish.csv")

In [34]:
example_data.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [35]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

32


In [36]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [37]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), num_columns)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Weight', 'Length1', 'Length2', 'Length3',
                                  'Height', 'Width'])])

In [38]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [39]:
# Устанавливаем названия столбцов
column_names = num_columns + ['Species']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [40]:
sc_train

,Weight,Length1,Length2,Length3,Height,Width,Species
0,0.30303,0.41165,0.410909,0.462838,0.747797,0.524222,Bream
1,0.075758,0.223301,0.229091,0.231419,0.234465,0.360201,Perch
2,0.0,0.223301,0.22,0.236486,0.280759,0.315031,Roach
3,0.005879,0.056311,0.047273,0.054054,0.027657,0.033116,Smelt
4,0.287879,0.405825,0.410909,0.462838,0.741373,0.565631,Bream
...,...,...,...,...,...,...,...
90,0.090909,0.252427,0.256364,0.256757,0.299497,0.353981,Perch
91,0.590909,0.580583,0.592727,0.626689,1.0,0.800575,Bream
92,0.424242,0.44466,0.447273,0.498311,0.77672,0.591541,Bream
93,0.042424,0.159223,0.163636,0.163851,0.169137,0.256392,Perch


In [41]:
# Задание №1 - анализ метода опорных векторов в задаче регрессии
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR
# kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'
# Только для kernel = 'poly' : degreeint, default=3

In [42]:
# Выбираем 4 числовых переменных, три их них будут предикторами, одна - зависимой переменной
n = 4
labels = random.sample(num_columns, n)

y_label = labels[0]
x_labels = labels[1:]

print(x_labels)
print(y_label)

['Length1', 'Height', 'Weight']
Length3


In [43]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [44]:
y_train


0     0.462838
1     0.231419
2     0.236486
3     0.054054
4     0.462838
        ...   
90    0.256757
91    0.626689
92    0.498311
93    0.163851
94    0.565878
Name: Length3, Length: 95, dtype: object

In [53]:
# Создайте 4 модели с различными ядрами: 'linear', 'poly', 'rbf', 'sigmoid'.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# При необходимости применяйте параметр регуляризации C : float, default=1.0
# Укажите, какая модель решает задачу лучше других.
r_model1 = SVR(kernel='linear')
r_model2 = SVR(kernel='poly')
r_model3 = SVR()
r_model4 = SVR(kernel='sigmoid')

In [54]:
r_model1.fit(x_train,y_train)

a=r_model1.predict(x_test)
mse1 = mean_squared_error(y_test, a)
mse1

0.0032019839231219828

In [55]:

r_model2.fit(x_train,y_train)
a=r_model2.predict(x_test)
mse2 = mean_squared_error(y_test, a)
mse2

0.003495030715050175

In [56]:
r_model3.fit(x_train,y_train)
a=r_model3.predict(x_test)
mse3 = mean_squared_error(y_test, a)
mse3

0.0019284788740379184

In [57]:
r_model4.fit(x_train,y_train)
a=r_model4.predict(x_test)
mse4 = mean_squared_error(y_test, a)
mse4

19.967515900525857

In [58]:
# Задание №2 - анализ метода опорных векторов в задаче классификации
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
# kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'
# Только для kernel = 'poly' : degreeint, default=3

In [59]:
# Выбираем 2 числовых переменных, которые будут параметрами элементов набора данных
# Метка класса всегда 'Species'
n = 2
x_labels = random.sample(num_columns, n)
y_label = 'Species'

print(x_labels)
print(y_label)

['Weight', 'Length3']
Species


In [60]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [61]:
y_train

0     Bream
1     Perch
2     Roach
3     Smelt
4     Bream
      ...  
90    Perch
91    Bream
92    Bream
93    Perch
94    Perch
Name: Species, Length: 95, dtype: object

In [62]:
# Создайте 4 модели с различными ядрами: 'linear', 'poly', 'rbf', 'sigmoid'.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# При необходимости применяйте параметр регуляризации C : float, default=1.0

c_model1 = SVC(kernel= 'linear',degree=1)
c_model2 = SVC(kernel='poly',degree=0.5)
c_model3 = SVC(degree=0)
c_model4 = SVC(kernel='sigmoid',degree=1)


In [63]:
c_model1.fit(x_train, y_train)
print(c_model1.predict(x_test))

['Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch']


In [64]:
c_model2.fit(x_train, y_train)
print(c_model2.predict(x_test))

['Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch']


In [65]:
c_model3.fit(x_train, y_train)
print(c_model3.predict(x_test))

['Smelt' 'Perch' 'Bream' 'Perch' 'Perch' 'Smelt' 'Bream' 'Perch' 'Bream'
 'Bream' 'Perch' 'Perch' 'Perch' 'Bream' 'Perch' 'Smelt' 'Perch' 'Perch'
 'Pike' 'Perch' 'Bream' 'Bream' 'Perch' 'Bream' 'Bream' 'Perch' 'Perch'
 'Smelt' 'Perch' 'Perch' 'Smelt' 'Perch']


In [66]:
c_model4.fit(x_train, y_train)
print(c_model4.predict(x_test))

['Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Smelt' 'Perch' 'Perch' 'Perch'
 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Smelt' 'Bream' 'Perch'
 'Perch' 'Bream' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Perch' 'Bream'
 'Perch' 'Bream' 'Bream' 'Perch' 'Bream']
